In [1]:
import os
from pycocotools.coco import COCO

# initialize COCO API for instance annotations
dataDir = ''
dataType = 'train2014'
instances_annFile = os.path.join(dataDir, 'coco/annotations/annotations/instances_train2014.json'.format(dataType))
coco = COCO(instances_annFile)

# initialize COCO API for caption annotations
captions_annFile = os.path.join(dataDir, 'coco/annotations/annotations/captions_train2014.json'.format(dataType))
coco_caps = COCO(captions_annFile)

# get image ids 
ids = list(coco.anns.keys())
len(ids)

ModuleNotFoundError: No module named 'pycocotools'

In [ ]:
# Load libraries
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
import random
from keras.preprocessing import image, sequence
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
%matplotlib inline

# pick a random image and obtain the corresponding URL
ann_id = np.random.choice(ids)
img_id = coco.anns[ann_id]['image_id']
img = coco.loadImgs(img_id)[0]
url = img['coco_url']
# print URL and visualize corresponding image
print(url)
I = io.imread(url)
plt.axis('off')
plt.imshow(I)
plt.show()

# load and display captions
annIds = coco_caps.getAnnIds(imgIds=img['id']);
print(annIds)
anns = coco_caps.loadAnns(annIds)
print(anns)
coco_caps.showAnns(anns)
anns[0]['caption']



In [ ]:
image_ids = {}
for i in ids:
    img_id = coco.anns[i]['image_id']
    if img_id in image_ids:
        continue
    else:
        image_ids[img_id]=i
        
len(image_ids)

In [ ]:
tokens = {}
x_train = []
unique = {}
c=1
for i in image_ids:
    img = coco.loadImgs(i)[0]
    file_name = img['file_name']
    annIds = coco_caps.getAnnIds(imgIds=img['id']);
    anns = coco_caps.loadAnns(annIds)
   # print(file_name)
    #tokens[file_name] = []
    for j in range(0,5):
        if file_name in tokens:
            tokens[file_name].append(anns[j]['caption'])
        else:
            x_train.append(file_name)
            tokens[file_name] = [anns[j]['caption']]
    
print(len(ids))
print(len(x_train))
print(c)
len(tokens['COCO_train2014_000000480023.jpg'])

In [ ]:
# Helper function to process images
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
# Loading 50 layer Residual Network Model and getting the summary of the model
from IPython.core.display import display, HTML
display(HTML("""<a href="http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006">ResNet50 Architecture</a>"""))
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')
model.summary()
# Note: For more details on ResNet50 architecture you can click on hyperlink given below

In [ ]:
train_data = {}
images_path = 'coco/coco-train/train2014/'
ctr=1
for ix in x_train:
    if ix == "":
        continue
    if ctr>= 82784:
        break
    #print(ix)
    path = images_path + ix
    img = preprocessing(path)
    pred = model.predict(img).reshape(2048)
    train_data[ix] = pred

In [ ]:
len(train_data)

In [ ]:
train_dataset = open('coco/coco-train/coco-train-dataset.txt','wb')
train_dataset.write(b"image_id\tcaptions\n")

In [ ]:
i=1
j=1
for img in x_train:
    if img == '':
        continue
    for capt in tokens[img]:
        if capt == "":
            continue
        if i==1:
            print(j)
            j=j+1
        caption = "<start> "+ capt + " <end>"
        train_dataset.write((img+"\t"+caption+"\n").encode())
        train_dataset.flush()
    i = i+1
train_dataset.close()

In [ ]:
# opening train_encoded_images.p file and dumping it's content
with open( "coco/train_encoded_images.p", "wb" ) as pickle_f:
    pickle.dump(train_data, pickle_f )

In [ ]:
# Loading image and its corresponding caption into a dataframe and then storing values from dataframe into 'ds'
pd_dataset = pd.read_csv("coco/coco-train/coco-train-dataset.txt", delimiter='\t')
pd_dataset = pd_dataset[pd_dataset['image_id'].notnull()]
ds = pd_dataset.values
#ds = ds[ds[i]!="" for i in ds]
print(ds.shape)

In [ ]:
pd_dataset.head()

In [ ]:
# Storing all the captions from ds into a list
sentences = []
for ix in range(ds.shape[0]):
    sentences.append(ds[ix, 1])
    
print(len(sentences))

In [ ]:
# First 5 captions stored in sentences
sentences[:5]

In [ ]:
# Splitting each captions stored in 'sentences' and storing them in 'words' as list of list
words = [str(i).split() for i in sentences]

In [ ]:
# Creating a list of all unique words
unique = []
for i in words:
    unique.extend(i)
unique = list(set(unique))

print(len(unique))

vocab_size = len(unique)

In [ ]:
# Vectorization
word_2_indices = {val:index for index, val in enumerate(unique)}
indices_2_word = {index:val for index, val in enumerate(unique)}
word_2_indices['UNK'] = 0
word_2_indices['raining'] = 43748
indices_2_word[0] = 'UNK'
indices_2_word[43748] = 'raining'
print(word_2_indices['<start>'])
print(indices_2_word[4011])
print(word_2_indices['<end>'])
print(indices_2_word[8051])

In [ ]:
vocab_size = len(word_2_indices.keys())
print(vocab_size)

In [ ]:
max_len = 0

for i in sentences:
    i = str(i).split()
    if len(i) > max_len:
        max_len = len(i)

print(max_len)

In [ ]:
ds.shape[0]

In [ ]:
padded_sequences, subsequent_words = [], []

for ix in range(ds.shape[0]):
    partial_seqs = []
    next_words = []
    text = str(ds[ix, 1]).split()
    text = [word_2_indices[i] for i in text]
   # print(text)
    for i in range(1, len(text)):
        partial_seqs.append(text[:i])
        next_words.append(text[i])
    padded_partial_seqs = sequence.pad_sequences(partial_seqs, max_len, padding='post')

    next_words_1hot = np.zeros([len(next_words), vocab_size], dtype=np.bool)
    
    #Vectorization
    for i,next_word in enumerate(next_words):
        next_words_1hot[i, next_word] = 1
        
    padded_sequences.append(padded_partial_seqs)
    subsequent_words.append(next_words_1hot)
    
padded_sequences = np.asarray(padded_sequences)
subsequent_words = np.asarray(subsequent_words)

print(padded_sequences.shape)
print(subsequent_words.shape)

In [ ]:
print(padded_sequences[0])

In [ ]:
for ix in range(len(padded_sequences[0])):
    for iy in range(max_len):
        print(indices_2_word[padded_sequences[0][ix][iy]],)
    print("\n")

print(len(padded_sequences[0]))

In [ ]:
num_of_images = 2000
captions = np.zeros([0, max_len])
next_words = np.zeros([0, vocab_size])